In [1]:
import torch
import torch.nn as nn
import torch.nn.parallel
from __future__ import print_function
import argparse
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable
import os

In [2]:
import cv2
import numpy as np
import functions
batchsize = 1

#X_train = bonemarrow_cell()
X_train = functions.get_mnist()
X_label = torch.LongTensor(np.zeros((X_train.shape[0]),dtype=int))
X_train = torch.FloatTensor(X_train)
train = torch.utils.data.TensorDataset(X_train,X_label)
train_loader = torch.utils.data.DataLoader(train, shuffle=True, batch_size=batchsize)

dataiter = iter(train_loader)

test,test_label = functions.get_test()
test = test[np.argsort(test_label)]
test_label = test_label[np.argsort(test_label)]

Test_label = torch.LongTensor(test_label.astype('int'))
Test = torch.FloatTensor(test)

test_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(Test,Test_label), shuffle=False, batch_size=64)

In [3]:
rand = 128
dis = 1

class _netG(nn.Module):
    def __init__(self, isize = 32, nz = 149, nc = 3, ngf = 64, n_extra_layers=0):
        super(_netG, self).__init__()
        assert isize % 16 == 0, "isize has to be a multiple of 16"

        cngf, tisize = ngf//2, 4
        while tisize != isize:
            cngf = cngf * 2
            tisize = tisize * 2

        main = nn.Sequential()
        # input is Z, going into a convolution
        main.add_module('initial.{0}-{1}.convt'.format(nz, cngf),
                        nn.ConvTranspose2d(nz, cngf, 4, 1, 0, bias=False))
        main.add_module('initial.{0}.batchnorm'.format(cngf),
                        nn.BatchNorm2d(cngf))
        main.add_module('initial.{0}.relu'.format(cngf),
                        nn.ReLU(True))

        csize, cndf = 4, cngf
        while csize < isize//2:
            main.add_module('pyramid.{0}-{1}.convt'.format(cngf, cngf//2),
                            nn.ConvTranspose2d(cngf, cngf//2, 4, 2, 1, bias=False))
            main.add_module('pyramid.{0}.batchnorm'.format(cngf//2),
                            nn.BatchNorm2d(cngf//2))
            main.add_module('pyramid.{0}.relu'.format(cngf//2),
                            nn.ReLU(True))
            cngf = cngf // 2
            csize = csize * 2

        # Extra layers
        for t in range(n_extra_layers):
            main.add_module('extra-layers-{0}.{1}.conv'.format(t, cngf),
                            nn.Conv2d(cngf, cngf, 3, 1, 1, bias=False))
            main.add_module('extra-layers-{0}.{1}.batchnorm'.format(t, cngf),
                            nn.BatchNorm2d(cngf))
            main.add_module('extra-layers-{0}.{1}.relu'.format(t, cngf),
                            nn.ReLU(True))

        main.add_module('final.{0}-{1}.convt'.format(cngf, nc),
                        nn.ConvTranspose2d(cngf, nc, 4, 2, 1, bias=False))
        main.add_module('final.{0}.tanh'.format(nc),
                        nn.Tanh())
        self.main = main

    def forward(self, input):
        return self.main(input)
    
netG = _netG(nz = rand+dis*10)
print (netG)


""" ==================== DISCRIMINATOR  ======================== """

class _netD(nn.Module):
    def __init__(self, isize = 32, nz = 149, nc = 3, ndf = 64, n_extra_layers=0):
        super(_netD, self).__init__()
        assert isize % 16 == 0, "isize has to be a multiple of 16"

        main = nn.Sequential()
        # input is nc x isize x isize
        main.add_module('initial.conv.{0}-{1}'.format(nc, ndf),
                        nn.Conv2d(nc, ndf, 4, 2, 1, bias=False))
        main.add_module('initial.relu.{0}'.format(ndf),
                        nn.LeakyReLU(0.2, inplace=True))
        csize, cndf = isize / 2, ndf

        # Extra layers
        for t in range(n_extra_layers):
            main.add_module('extra-layers-{0}.{1}.conv'.format(t, cndf),
                            nn.Conv2d(cndf, cndf, 3, 1, 1, bias=False))
            main.add_module('extra-layers-{0}.{1}.batchnorm'.format(t, cndf),
                            nn.BatchNorm2d(cndf))
            main.add_module('extra-layers-{0}.{1}.relu'.format(t, cndf),
                            nn.LeakyReLU(0.2, inplace=True))

        while csize > 4:
            in_feat = cndf
            out_feat = cndf * 2
            main.add_module('pyramid.{0}-{1}.conv'.format(in_feat, out_feat),
                            nn.Conv2d(in_feat, out_feat, 4, 2, 1, bias=False))
            main.add_module('pyramid.{0}.batchnorm'.format(out_feat),
                            nn.BatchNorm2d(out_feat))
            main.add_module('pyramid.{0}.relu'.format(out_feat),
                            nn.LeakyReLU(0.2, inplace=True))
            cndf = cndf * 2
            csize = csize / 2

        # state size. K x 4 x 4
       # main.add_module('final.{0}-{1}.conv'.format(cndf, 1),
       #                 nn.Conv2d(cndf, 1, 4, 1, 0, bias=False))
        self.main = main
        
    def forward(self, input):
        return self.main(input)


netD = _netD(nz = rand+dis*10)
print (netD)

_netG (
  (main): Sequential (
    (initial.138-256.convt): ConvTranspose2d(138, 256, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (initial.256.batchnorm): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
    (initial.256.relu): ReLU (inplace)
    (pyramid.256-128.convt): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (pyramid.128.batchnorm): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True)
    (pyramid.128.relu): ReLU (inplace)
    (pyramid.128-64.convt): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (pyramid.64.batchnorm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
    (pyramid.64.relu): ReLU (inplace)
    (final.64-3.convt): ConvTranspose2d(64, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (final.3.tanh): Tanh ()
  )
)
_netD (
  (main): Sequential (
    (initial.conv.3-64): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bi

In [4]:
class _netD_D(nn.Module):
    def __init__(self):
        super(_netD_D, self).__init__()
        self.conv = nn.Conv2d(256, 1, 4, 1, 0, bias=False)
        
    def forward(self, x):
        x = self.conv(x)
        return x
    
class _netD_Q(nn.Module):
    def __init__(self, nd = 10):
        super(_netD_Q, self).__init__()
        # input is Z, going into a convolution
        self.conv = nn.Conv2d(256, 10, 4, 1, 0, bias=False)
        #self.softmax = nn.LogSoftmax()

    def forward(self, x):
        x = self.conv(x)
        #x = self.softmax(x)
       # x = x.view(64,10)
        return x

netD_D = _netD_D()
netD_Q = _netD_Q()

In [5]:
netD, netG, netD_D, netD_Q = netD.cuda(), netG.cuda(), netD_D.cuda(), netD_Q.cuda()

def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

netG.apply(weights_init)
netD.apply(weights_init)
netD_Q.apply(weights_init)
netD_D.apply(weights_init)

_netD_D (
  (conv): Conv2d(256, 1, kernel_size=(4, 4), stride=(1, 1), bias=False)
)

In [6]:
optimizerD = optim.RMSprop([
                {'params': netD.parameters()},
                {'params': netD_D.parameters()}
            ], 0.00005)

optimizerG = optim.RMSprop(netG.parameters(), lr = 0.00005)
 
optimizerQ = optim.RMSprop([
                {'params': netG.parameters()},            
                {'params': netD.parameters()},
                {'params': netD_Q.parameters()},
            ], 0.00005)

input = torch.FloatTensor(batchsize, 3, 32, 32)
noise = torch.FloatTensor(batchsize, rand+10*dis,1 ,1 )

fixed_noise = torch.FloatTensor(np.random.multinomial(batchsize, 10*[0.1], size=1))
c = torch.randn(batchsize, 10)
z = torch.randn(batchsize, rand)

label = torch.FloatTensor(1)

real_label = 1
fake_label = 0

criterion = nn.BCELoss()
criterion_logli = nn.NLLLoss()
criterion_mse = nn.MSELoss()

criterion, criterion_logli, criterion_mse = criterion.cuda(), criterion_logli.cuda(), criterion_mse.cuda()
input, label = input.cuda(), label.cuda()
noise, fixed_noise = noise.cuda(), fixed_noise.cuda()
z, c = z.cuda(), c.cuda()

In [13]:
loss_c = -np.ones((1,10),dtype='float32')
loss_c

array([[-1., -1., -1., -1., -1., -1., -1., -1., -1., -1.]], dtype=float32)

In [23]:
def sample_c(batchsize):
    rand_c = np.zeros((batchsize,10),dtype='float32')
    for i in range(0,batchsize):
        rand = np.random.multinomial(1, 10*[0.1], size=1)
        rand_c[i] = rand
    label_c = np.argmax(rand_c,axis=1)
    loss_c = -np.ones((batchsize,10),dtype='float32')
    loss_c[0,label_c] = 1
    label_c = torch.LongTensor(label_c.astype('int'))
    rand_c = torch.from_numpy(rand_c.astype('float32'))
    loss_c = torch.FloatTensor(loss_c.astype('float32'))
    return rand_c,loss_c

def zero_grad():
    netD.zero_grad()
    netD_Q.zero_grad()
    netD_D.zero_grad()
    netG.zero_grad()

def weight_clamp():
    for p in netD.parameters():
            p.data.clamp_(-0.01, 0.01)
    for p in netD_D.parameters():
            p.data.clamp_(-0.01, 0.01)
    for p in netD_Q.parameters():
            p.data.clamp_(-0.01, 0.01)
    #for p in netD_Q_2.parameters():
            #p.data.clamp_(-0.01, 0.01)
    #for p in netD_Q_3.parameters():
            #p.data.clamp_(-0.01, 0.01)
        
def generate_fix_noise(dis=1, cont=4, rand=128):
    
    fixed_z = np.random.randn(10,rand).repeat(10,axis=0)
    changing_dis = np.zeros((100,10),dtype = np.float32)
    list = [n for n in range(0,10)]*10
    for i in range(0,100):
        changing_dis[i,list[i]] = 1
    fixed_cont = np.zeros((100,cont),dtype = np.float32)
    map1 = np.concatenate((changing_dis,fixed_cont,fixed_z),axis=1)
    
    lst = [map1.astype(np.float32)]
    single_cont = np.asarray([float(n-5)*2/5 for n in range(0,10)]*10,dtype = np.float32)
    
    fixed_dis = np.zeros((100,10),dtype=np.float32)
    for t in range(0,5):
        fixed_dis[t*20:t*20+20,t*2] = 1
        
    for t in range (0,4):
        fixed_cont = np.zeros((100,cont),dtype = np.float32)
        fixed_cont[:,t] = single_cont
        map2 = np.concatenate((fixed_dis,fixed_cont,fixed_z),axis=1)
        lst.append(map2.astype(np.float32))
    
    return lst

def fix_noise(dis=1, rand=128):
        
    fixed_z = np.random.randn(10,rand).repeat(10,axis=0)
    changing_dis = np.zeros((100,10),dtype = np.float32)
    list = [n for n in range(0,10)]*10
    for i in range(0,100):
        changing_dis[i,list[i]] = 1
    map1 = np.concatenate((changing_dis,fixed_z),axis=1)
    
    lst = [map1.astype(np.float32)]
    return lst[0].reshape(100,138,1,1)


one = torch.FloatTensor([1])
mone = one * -1
one = one.cuda()
mone = mone.cuda()
fixed_noise = torch.from_numpy(fix_noise()).cuda()

In [ ]:
gen_iterations = 0

for epoch in range(100000):

    dataiter = iter(train_loader)
    i = 0
    
    while i < len(train_loader):
        weight_clamp()
        if gen_iterations < 25 or gen_iterations % 500 == 0:
            Diters = 1
        else:
            Diters = 1
        
        j = 0
        while j < Diters and i < len(train_loader):
            j += 1
            image_, _ = dataiter.next()
            _batchsize = image_.size(0)
            
            image_ = image_.cuda()
            
            i +=1
            weight_clamp()
    #train on D
    #sending real data 
            zero_grad()
            input.resize_as_(image_).copy_(image_)
            inputv = Variable(input)
         #   label.data.resize_(1).fill_(real_label)
            D_real =netD_D(netD(inputv)).mean(0).view(1)
            #D_loss_real = criterion(D_real, label)
            D_real.backward(one)

    #sending noise
            z.normal_(0, 1)
            rand_c,loss_c = sample_c(batchsize)
            c.copy_(rand_c)
            noise = torch.cat([c,z],1)
            noise_resize = noise.view(batchsize,rand+10*dis,1,1)
            noisev = Variable(noise_resize)
            
            G_sample = Variable(netG(noisev).data)
            inputv = G_sample
            D_fake = netD_D(netD(inputv)).mean(0).view(1)
         #   label.data.resize_(1).fill_(fake_label)
           # D_loss_fake = criterion(D_fake, label)
            D_fake.backward(mone)
        
    # update D
            optimizerD.step()
    
        for p in netD.parameters():
            p.requires_grad = False # to avoid computation
        for p in netD_D.parameters():
            p.requires_grad = False # to avoid computation

    # update G  
        zero_grad()
        noisev = Variable(noise_resize)
        G_sample = netG(noisev)
        D_fake = netD_D(netD(G_sample)).mean(0).view(1)
      #  label.data.resize_(1).fill_(real_label)
       # G_loss = criterion(D_fake, label)
        D_fake.backward(one)
        optimizerG.step()
        
        gen_iterations += 1
        
        for p in netD.parameters():
            p.requires_grad = True # to avoid computation
        for p in netD_D.parameters():
            p.requires_grad = True # to avoid computation

    # update Q
        zero_grad()
        noisev = Variable(noise_resize)
        G_sample = netG(noisev)
        Q_c_given_x = netD_Q(netD(G_sample)).view(batchsize, 10)
        Q_c_given_x.backward(loss_c.cuda()*0.1)
        #crossent_loss = criterion_logli(Q_c_given_x ,Variable(label_c.cuda()))
       # print (Q_c_given_x)

        # ent_loss = torch.mean(-torch.sum(c * torch.log(c + 1e-8), dim=1))
       # ent_loss_2 = torch.mean(-torch.sum(c2 * torch.log(c2 + 1e-8), dim=1))
       # ent_loss_3 = torch.mean(-torch.sum(c3 * torch.log(c3 + 1e-8), dim=1))

        #mi_loss = 1*crossent_loss

        #mi_loss.backward()
        optimizerQ.step()
        
        if gen_iterations % 20 == 0 :
            errD = D_real - D_fake
            #with open("output_cell.txt","w") as f:
                #f.write('{0} {1} {2} {3}'.format(epoch, gen_iterations , -errD.data[0] , mi_loss.data[0]) + '\n')
            print ('{0} {1} {2} {3}'.format(epoch, gen_iterations , -errD.data[0] , Q_c_given_x.data.mean()))
            G_sample = netG(Variable(fixed_noise))
            vutils.save_image(G_sample.data, 'fake_EM.png',nrow=10,normalize=True)
            
            #vutils.save_image(G_sample.data, '{0}fake_samples_{1}.png'.format(-errD.data[0], gen_iterations))
            #vutils.save_image(G_sample.data, 'fake_samples.png',normalize = True)
            
            #for t in range(0,5):
                #fixed_noise = generate_fix_noise(dis, cont, rand)[t].reshape(100,rand+dis*10+cont,1,1)
                #G_sample = netG(Variable(torch.FloatTensor(fixed_noise).cuda()))
                #vutils.save_image(G_sample.data, 'map_%d_cell.png'% (t),nrow=10,normalize=True)

            #torch.save(netG.state_dict(), './params/0517/tumor_netG_epoch_%d.pth' % (epoch))
            #torch.save(netD.state_dict(), './params/0517/tumor_netD_epoch_%d.pth' % (epoch))
            #torch.save(netD_D.state_dict(), './params/0517/tumor_netD_D_epoch_%d.pth' % (epoch))
            #torch.save(netD_Q.state_dict(), './params/0517/tumor_netD_Q_epoch_%d.pth' % (epoch))
            #torch.save(netD_Q_2.state_dict(), './params/0517/tumor_netD_Q_2_epoch_%d.pth' % (epoch))
            #torch.save(netD_Q_3.state_dict(), './params/0517/tumor_netD_Q_3_epoch_%d.pth' % (epoch))
        
        #storage = np.zeros((100,3,64,64),dtype=np.float32)
        #z_fix = Variable(torch.randn(1,128,1,1).cuda().normal_(0, 1))

0 20 0.588555812836 -2.30262827873
0 40 0.580922722816 -2.30263209343
0 60 0.450013756752 -2.30263733864
0 80 0.543458223343 -2.30264854431
0 100 0.426252782345 -2.3028049469
0 120 0.426111727953 -2.30270147324
0 140 0.433038473129 -2.30271267891
0 160 0.491867959499 -2.30286717415
0 180 0.531347870827 -2.30280184746
0 200 0.429466485977 -2.30276489258
0 220 0.467031180859 -2.30282402039
0 240 0.467104494572 -2.30279803276
0 260 0.494022488594 -2.3028986454
0 280 0.466758489609 -2.30277872086
0 300 0.464911609888 -2.30273962021
0 320 0.489356100559 -2.30276918411
0 340 0.429176211357 -2.30279493332
0 360 0.278278529644 -2.30277824402
0 380 0.465960115194 -2.30269837379
0 400 0.419904947281 -2.30282402039
0 420 0.47141546011 -2.30295801163
0 440 0.297507286072 -2.30287218094
0 460 0.482081204653 -2.30304551125
0 480 0.429213762283 -2.30286741257
0 500 0.523908138275 -2.30286192894
0 520 0.517455756664 -2.30285310745
0 540 0.333028495312 -2.3028280735
0 560 0.457054257393 -2.30288004875


0 4520 0.418548882008 -2.30344438553
0 4540 0.328580617905 -2.30319833755
0 4560 0.22844837606 -2.30331754684
0 4580 0.329023033381 -2.30348300934
0 4600 0.321020364761 -2.30392360687
0 4620 0.253313302994 -2.30328702927
0 4640 0.428055346012 -2.30324006081
0 4660 0.230958640575 -2.30376005173
0 4680 0.428944498301 -2.30373692513
0 4700 0.181774586439 -2.30307722092
0 4720 0.417638361454 -2.30336141586
0 4740 0.154258981347 -2.30487990379
0 4760 0.244891613722 -2.30324482918
0 4780 0.463790118694 -2.30456066132
0 4800 0.509078085423 -2.30272960663
0 4820 0.247135356069 -2.30346822739
0 4840 0.375588148832 -2.30296397209
0 4860 0.429126739502 -2.30375599861
0 4880 0.289926886559 -2.30336928368
0 4900 0.341878414154 -2.30309295654
0 4920 0.336784899235 -2.3031475544
0 4940 0.144288957119 -2.30353307724
0 4960 0.109877459705 -2.30296397209
0 4980 0.457834124565 -2.3030192852
0 5000 0.383752584457 -2.30281329155
0 5020 0.362591207027 -2.30288529396
0 5040 0.0938324555755 -2.30373263359
0 5

0 8980 0.312517225742 -2.30789637566
0 9000 0.375065356493 -2.30735516548
0 9020 0.466823637486 -2.30679631233
0 9040 0.221206650138 -2.30684399605
0 9060 0.282417714596 -2.31069469452
0 9080 0.222800791264 -2.31261777878
0 9100 0.371816813946 -2.30572748184
0 9120 0.250672757626 -2.30673861504
0 9140 0.222678869963 -2.30668568611
0 9160 0.403310239315 -2.30835175514
0 9180 0.201886504889 -2.30549716949
0 9200 0.300908118486 -2.30752491951
0 9220 0.395770579576 -2.30459332466
0 9240 0.242987692356 -2.31004524231
0 9260 0.442393362522 -2.30730223656
0 9280 0.353217154741 -2.31230926514
0 9300 0.166605561972 -2.31298780441
0 9320 0.185888782144 -2.30804109573
0 9340 0.307229757309 -2.30554056168
0 9360 0.258674830198 -2.30743861198
0 9380 0.283908843994 -2.30595541
0 9400 0.376147568226 -2.31007742882
0 9420 0.318502306938 -2.30656599998
0 9440 0.214520096779 -2.30824065208
0 9460 0.393913626671 -2.31330752373
0 9480 0.257394403219 -2.30564665794
0 9500 0.207946911454 -2.31068372726
0 95

0 13360 0.343136966228 -2.31278085709
0 13380 0.24780215323 -2.31193089485
0 13400 0.294336825609 -2.31103944778
0 13420 0.368295252323 -2.31390285492
0 13440 0.198764890432 -2.31162977219
0 13460 0.323519766331 -2.31370639801
0 13480 0.21447750926 -2.31104230881
0 13500 0.348111450672 -2.30708932877
0 13520 0.285411536694 -2.30843758583
0 13540 0.185845002532 -2.30465722084
0 13560 0.220362111926 -2.31248354912
0 13580 0.250503867865 -2.30896663666
0 13600 0.455373555422 -2.31178617477
0 13620 0.24645344913 -2.3087182045
0 13640 0.225166112185 -2.30824923515
0 13660 0.224346518517 -2.31409573555
0 13680 0.437725782394 -2.3129992485
0 13700 0.140819430351 -2.31119990349
0 13720 0.300412088633 -2.31096696854
0 13740 0.11206574738 -2.30779719353
0 13760 0.307329237461 -2.31208562851
0 13780 0.432385951281 -2.31133079529
0 13800 0.331483364105 -2.30957245827
0 13820 0.175689890981 -2.30876278877
0 13840 0.185455679893 -2.31225681305
0 13860 0.224055752158 -2.30726003647
0 13880 0.36397111

0 17700 0.240563824773 -2.30647921562
0 17720 0.251083940268 -2.31335115433
0 17740 0.151307791471 -2.3101170063
0 17760 0.248834073544 -2.3106341362
0 17780 0.372559338808 -2.30666470528
0 17800 0.217341333628 -2.30948090553
0 17820 0.342885494232 -2.31199598312
0 17840 0.375030219555 -2.31461334229
0 17860 0.290049314499 -2.31192231178
0 17880 0.207710921764 -2.31240129471
0 17900 0.258926749229 -2.31154131889
0 17920 0.21683986485 -2.31289339066
0 17940 0.256722569466 -2.31044173241
0 17960 0.275095760822 -2.31335711479
0 17980 0.100243881345 -2.30847811699
0 18000 0.297504067421 -2.30915331841
0 18020 0.327830374241 -2.31129407883
0 18040 0.393754929304 -2.31190896034
0 18060 0.253871917725 -2.31445240974
0 18080 0.221457317472 -2.30756902695
0 18100 0.338628947735 -2.31274485588
0 18120 0.191488772631 -2.30981564522
0 18140 0.279190123081 -2.31051445007
0 18160 0.307438373566 -2.31208062172
0 18180 0.172275170684 -2.3085706234
0 18200 0.257072627544 -2.30993652344
0 18220 0.175072

...

2 159580 0.313620835543 -2.31326913834
2 159600 0.235317766666 -2.31093645096
2 159620 0.220088616014 -2.3124666214
2 159640 0.163236916065 -2.31062245369
2 159660 0.348493784666 -2.31226539612
2 159680 0.182967871428 -2.3118596077
2 159700 0.205426201224 -2.31013846397
2 159720 0.24384893477 -2.310567379
2 159740 0.223603740335 -2.31087040901
2 159760 0.255310595036 -2.31243181229
2 159780 0.314660012722 -2.30999660492
2 159800 0.148745134473 -2.31070661545
2 159820 0.303468465805 -2.31226205826
2 159840 0.259292006493 -2.31089305878
2 159860 0.230064362288 -2.30994462967
2 159880 0.192927330732 -2.31317543983
2 159900 0.274111688137 -2.31284666061
2 159920 0.166287943721 -2.31183218956
2 159940 0.233596339822 -2.31299948692
2 159960 0.200182199478 -2.30998468399
2 159980 0.216528639197 -2.31230974197
2 160000 0.257709264755 -2.31310915947
2 160020 0.173833757639 -2.31048727036
2 160040 0.247251242399 -2.31194925308
2 160060 0.283892840147 -2.31325793266
2 160080 0.233816713095 -2.310

2 163820 0.173109352589 -2.31065225601
2 163840 0.298259913921 -2.31242823601
2 163860 0.190727099776 -2.31215786934
2 163880 0.251155257225 -2.31257534027
2 163900 0.185337483883 -2.30974173546
2 163920 0.271178692579 -2.31324362755
2 163940 0.161287739873 -2.31314611435
2 163960 0.281798154116 -2.31102252007
2 163980 0.301979541779 -2.3122856617
2 164000 0.244569554925 -2.31208777428
2 164020 0.295458555222 -2.31230783463
2 164040 0.308101713657 -2.31227684021
2 164060 0.134354501963 -2.31009340286
2 164080 0.244214206934 -2.31284379959
2 164100 0.168801113963 -2.31245064735
2 164120 0.257651567459 -2.31281518936
2 164140 0.284909307957 -2.31114840508
2 164160 0.221288070083 -2.31211304665
2 164180 0.379322946072 -2.3127822876
2 164200 0.205756559968 -2.31340241432
2 164220 0.234098091722 -2.31218671799
2 164240 0.281572580338 -2.31149792671
2 164260 0.245149418712 -2.30990171432
2 164280 0.154159188271 -2.30952215195
2 164300 0.0887525379658 -2.31294441223
2 164320 0.212259739637 -2

2 168060 0.218480572104 -2.31044626236
2 168080 0.238116592169 -2.31310796738
2 168100 0.191815361381 -2.31034731865
2 168120 0.238884329796 -2.31054234505
2 168140 0.316836923361 -2.3111770153
2 168160 0.317267179489 -2.31317400932
2 168180 0.255761355162 -2.31054878235
2 168200 0.208006933331 -2.31192874908
2 168220 0.183768779039 -2.31192970276
2 168240 0.22191670537 -2.31394743919
2 168260 0.271135747433 -2.31231236458
2 168280 0.312344521284 -2.31305193901
2 168300 0.16057138145 -2.31000089645
2 168320 0.190411061049 -2.31322169304
2 168340 0.165453910828 -2.31371021271
2 168360 0.231315478683 -2.31048727036
2 168380 0.155171349645 -2.31346821785
2 168400 0.223930642009 -2.31266641617
2 168420 0.26617193222 -2.31065058708
2 168440 0.290142059326 -2.3117556572
2 168460 0.215685412288 -2.31145167351
2 168480 0.253569275141 -2.31033158302
2 168500 0.285912334919 -2.31251716614
2 168520 0.278843224049 -2.31060862541
2 168540 0.194989845157 -2.31344652176
2 168560 0.202215507627 -2.313

2 172300 0.286404371262 -2.31195139885
2 172320 0.185524553061 -2.31228756905
2 172340 0.187081456184 -2.31008934975
2 172360 0.264822423458 -2.31071829796
2 172380 0.163287997246 -2.30999803543
2 172400 0.251916348934 -2.31207561493
2 172420 0.182224109769 -2.31212472916
2 172440 0.255575746298 -2.31024956703
2 172460 0.272997528315 -2.31180787086
2 172480 0.268326252699 -2.31129431725
2 172500 0.230927437544 -2.31164669991
2 172520 0.212283924222 -2.31225919724
2 172540 0.153755426407 -2.31231641769
2 172560 0.267472475767 -2.31237459183
2 172580 0.266343176365 -2.31228780746
2 172600 0.269388735294 -2.31220984459
2 172620 0.340636193752 -2.31289100647
2 172640 0.286872982979 -2.31246209145
2 172660 0.253089874983 -2.31082963943
2 172680 0.298514425755 -2.31092262268
2 172700 0.207900434732 -2.31266641617
2 172720 0.29109197855 -2.31291389465
2 172740 0.241072803736 -2.31286025047
2 172760 0.222021728754 -2.31028866768
2 172780 0.221483081579 -2.31235980988
2 172800 0.15307790041 -2.

2 176540 0.18978728354 -2.31256270409
2 176560 0.0855542570353 -2.3098077774
2 176580 0.270675569773 -2.31204628944
2 176600 0.192768633366 -2.3126718998
2 176620 0.197412699461 -2.31009054184
2 176640 0.244345471263 -2.31079626083
2 176660 0.194184228778 -2.31318330765
2 176680 0.205974757671 -2.31152653694
2 176700 0.294631332159 -2.31272268295
2 176720 0.231971085072 -2.31366968155
2 176740 0.182744517922 -2.31073093414
2 176760 0.260506868362 -2.3122484684
2 176780 0.262160241604 -2.31052207947
2 176800 0.240237265825 -2.31259846687
2 176820 0.27913081646 -2.31177282333
2 176840 0.249319463968 -2.31234431267
2 176860 0.147305041552 -2.31317949295
2 176880 0.235138103366 -2.3118083477
2 176900 0.221665471792 -2.31183624268
2 176920 0.265871942043 -2.31227946281
2 176940 0.154133930802 -2.3096871376
2 176960 0.227258846164 -2.31235051155
2 176980 0.266000896692 -2.31329631805
2 177000 0.238583654165 -2.30994844437
2 177020 0.204135894775 -2.31025481224
2 177040 0.190512314439 -2.3118

In [10]:
D_real - D_fake

Variable containing:
 0.2353
[torch.cuda.FloatTensor of size 1 (GPU 0)]